In [57]:
import pandas as pd 
from datetime import datetime, timedelta, date
import numpy as np

In [58]:
def porcentaje(asignadas, realizados):
    return (sum(realizados)/sum(asignadas))*100

In [59]:
tickets = pd.read_excel(r"C:\Users\centro.control\Downloads\Tickets de Recursos Humanos (CTS) (respuestas) - 2025-10-15T084808.721.xlsx")
rutinas = pd.read_excel(r"C:\Users\centro.control\Downloads\CUMPLIMIENTO DE RUTINAS - 2025-10-15T084850.217.xlsx", header=1)
vacantes = pd.read_excel(r"C:\Users\centro.control\Desktop\Monitor CMC\Actualizacion de informacion\Vacantes 2025.xlsx")
ub = pd.read_excel(r"C:\Users\centro.control\Downloads\activos-universida-bennetts15-10-2025_8_49am.xlsx")

# TICKETS

In [60]:
tickets['Fecha compromiso de cobertura']= pd.to_datetime(tickets['Fecha compromiso de cobertura'], errors='coerce').dt.date
tickets['Marca temporal']= pd.to_datetime(tickets['Marca temporal'], errors='coerce').dt.date
tickets['Fecha de cierre']= pd.to_datetime(tickets['Fecha de cierre'], errors='coerce').dt.date
hoy = (datetime.today()).date()
ayer = (datetime.today()-timedelta(days=1)).date()
antiayer = (datetime.today()-timedelta(days=2)).date()
anti_antiayer = (datetime.today()-timedelta(days=3)).date()
mañana = (datetime.today()+timedelta(days=1)).date()
pasado_mañ = (datetime.today()+timedelta(days=2)).date()
trasmañana = (datetime.today()+timedelta(days=3)).date()
ticket_abiertos_proceden = tickets[
    (tickets['Estatus'] == 'Abierto') &
    (tickets['Procede'] == 'Procede') &
    (tickets['Marca temporal'] > datetime.strptime('2025-01-01', '%Y-%m-%d').date())
]

ticket_abiertos = tickets[tickets['Estatus']== 'Abierto']  





ticket_cerrados = sum(tickets['Fecha de cierre'] == hoy)+ sum(tickets['Fecha de cierre'] == ayer)  #+ sum(tickets['Marca temporal'] == antiayer)+ sum(tickets['Marca temporal'] == anti_antiayer)
ticket_nuevos = sum(tickets['Marca temporal'] == hoy)+ sum(tickets['Marca temporal'] == ayer)  #+ sum(tickets['Marca temporal'] == antiayer) 
ticket_xvencer = sum(ticket_abiertos['Fecha compromiso de cobertura'] == mañana) + sum(ticket_abiertos['Fecha compromiso de cobertura'] == pasado_mañ) + sum(ticket_abiertos['Fecha compromiso de cobertura'] == trasmañana)
tickets_abiertos,_ =ticket_abiertos_proceden.shape



# CUMPLIMIENTO DE RUTINAS

In [61]:

rutinas = rutinas.dropna(subset=['ID Cedula'])
total_rutinas =round(porcentaje(rutinas['Rutinas Asignadas'],rutinas['Rutinas Realizadas']),1)


In [62]:
print(f'{ticket_cerrados}')
print(f'{ticket_nuevos}')
print(f'{ticket_xvencer}')
print(f'{tickets_abiertos}')
print(f'{total_rutinas}')

4
7
1
37
99.4


# VACANTES

In [63]:
vacantes

,id,Fecha del reporte,Sitio,Plantilla,Activos,Vacante,Turno,Tipo,Ejecutivo
0,241,2025-03-01,LIVERPOOL PERINORTE,14,11,3,Matutino,Natural,Karina Sánchez
1,358,2025-03-01,AMERICAN EXPRESS POLANCO,14,15,0,Matutino,Natural,Ángel Cervantes
2,357,2025-03-01,LIVERPOOL JILOTEPEC,13,13,0,Matutino,Natural,Oscar Castillo
3,183,2025-03-01,LIVERPOOL INTERLOMAS,27,27,0,Matutino,Natural,Abel Vázquez
4,80,2025-03-01,CUMBRES PRIMARIA Y KINDER,7,8,0,Matutino,Natural,Ángel Cervantes
...,...,...,...,...,...,...,...,...,...
91843,95,2025-10-14,TELEMUNDO,0,0,0,Mixto,NATURAL,Guillermina Vázquez
91844,93,2025-10-14,SOFTTEK POLANCO,0,0,0,Mixto,NATURAL,Karla Gómez
91845,78,2025-10-14,PM STEELE,0,0,0,Mixto,NATURAL,Ángel Cervantes
91846,67,2025-10-14,MCCANN,0,0,0,Mixto,NATURAL,Ángel Cervantes


In [64]:
duplicados = vacantes.groupby(['Sitio', 'Turno', 'Fecha del reporte']).size()
duplicados = duplicados[duplicados > 1]
print(duplicados)

Series([], dtype: int64)


In [65]:

vacantes['Fecha del reporte'] = pd.to_datetime(vacantes['Fecha del reporte'])
vacantes['Fecha del reporte'] = vacantes['Fecha del reporte'].dt.date
vacantes = vacantes[['Sitio','Turno','Fecha del reporte','Vacante']]
tabla_pivot = vacantes.pivot(
    index=['Sitio', 'Turno'],
    columns='Fecha del reporte',
    values='Vacante', 
)

tabla_pivot = tabla_pivot.reset_index()
tabla_pivot = tabla_pivot.fillna(0)
fechas = tabla_pivot.columns[2:]

In [66]:
total_vacantes = np.sum(tabla_pivot.iloc[:,-1])
total_vacantes = int(total_vacantes)

In [67]:
vacantes_cubiertas = {}
for i in range(len(fechas) - 1):
    vacantes_cubiertas[f'cubiertas: {fechas[i]} - {fechas[i+1]}'] = (
        tabla_pivot[fechas[i]] - tabla_pivot[fechas[i+1]]
    ).clip(lower=0)

vacantes_nuevas = {}
for i in range(len(fechas) - 1):
    vacantes_nuevas[f'Nuevas: {fechas[i+1]} - {fechas[i]}'] = (
        tabla_pivot[fechas[i+1]] - tabla_pivot[fechas[i]]
    ).clip(lower=0)

sinmov = {}
for i in range(len(fechas) - 1):
    sinmov[f'Sin movimiento: min({fechas[i+1]},{fechas[i]})'] = tabla_pivot[[fechas[i+1], fechas[i]]].min(axis=1)

In [68]:
vacantes_cubiertas = pd.DataFrame(vacantes_cubiertas).fillna(0)
vacantes_cubiertas_totales = int(np.sum(np.array(vacantes_cubiertas.iloc[:,-1])))
vacantes_nuevas = pd.DataFrame(vacantes_nuevas).fillna(0)
vacantes_nuevas_totales = int(np.sum(np.array(vacantes_nuevas.iloc[:,-1])))
sinmov = pd.DataFrame(sinmov).fillna(0)
vacantes_sinmov_totales= int(np.sum(np.array(sinmov.iloc[:,-1])))

In [69]:
print(vacantes_cubiertas_totales)
print(vacantes_nuevas_totales)
print(vacantes_sinmov_totales)
print(total_vacantes)

0
10
22
32


In [70]:
def contar_no_ceros_consecutivos(fila):
    contador = 0
    for val in reversed(fila):
        if val != 0:
            contador += 1
        else:
            break
    return contador

conteos = sinmov.apply(contar_no_ceros_consecutivos, axis=1)


conteos= [x+1 if x!=0 else 0 for x in conteos ]


sinmov['Dias con vacantes']=conteos
sinmov['Turno'] = tabla_pivot['Turno']
sinmov['Sitios'] = tabla_pivot['Sitio']
resumen=sinmov[['Sitios','Turno', 'Dias con vacantes']].sort_values(by='Dias con vacantes', ascending= False)

vacantes_top5 = resumen.head(5)

C:\Users\centro.control\AppData\Local\Temp\ipykernel_27572\4116121981.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for val in reversed(fila):


In [71]:
html_vacantes = "\n".join(
    f"<li>{row['Sitios']} ({row['Turno']}): <strong>{row['Dias con vacantes']} días</strong></li>"
    for _, row in vacantes_top5.iterrows()
)


# UB

In [72]:

ub_bajos= ub[['sitio','turno', 'Unnamed: 14']]
ub_certificados = ub.groupby(['sitio', 'turno']).agg(
    total = ('Unnamed: 14', 'count'),
    Certificados= ('Unnamed: 14','sum')
)

ub_certificados["% de certificados"] = ((ub_certificados['Certificados']*100/ub_certificados['total'])).round(2)
ub_certificados = ub_certificados.reset_index()
ub_certificados=ub_certificados[(ub_certificados['sitio'] != 'CAPACITADORES BENNETTS') & (ub_certificados['sitio'] != 'SHEIN') &
(ub_certificados['sitio'] != 'CCGENERI PRE ALTAS') & (ub_certificados['sitio'] != 'CONTINNUO EVENTOS') & (ub_certificados['sitio'] != 'MEDICINA METABOLICA')]


top_5_ub = ub_certificados[['sitio','turno',"% de certificados"]].sort_values(by="% de certificados").head(5)

In [73]:
top_5_ub

,sitio,turno,% de certificados
53,CL01601 DESPEGAR,Matutino,0.0
89,CL02901 KRISPY KREME,Vespertino,0.0
88,CL02901 KRISPY KREME,Nocturno,0.0
205,PLANET FITNESS ACOXPA,Nocturno,0.0
209,PLANET FITNESS MIXCOAC,Vespertino,0.0


In [74]:
html_ub = "\n".join(
    f"<li>{filas['sitio']} ({filas['turno']}): <strong>{filas['% de certificados']} %</strong></li>"
    for _, filas in top_5_ub.iterrows()
)

# Importar a HTML

In [75]:
# Leer tu archivo HTML base
with open("C:\\Users\\centro.control\\Documents\\code (1) - copia.html", "r", encoding="utf-8") as f:
    html = f.read()

# Reemplazar los marcadores por los valores
html = html.replace("{{ticket_abiertos}}", str(tickets_abiertos))
html = html.replace("{{ticket_nuevos}}", str(ticket_nuevos))
html = html.replace("{{ticket_cerrados}}", str(ticket_cerrados))
html = html.replace("{{ticket_xvencer}}", str(ticket_xvencer))
html = html.replace("{{%_de_Rutinas}}", str(total_rutinas)+"%")
html = html.replace("{{lista_vacantes}}", html_vacantes)
html = html.replace("{{lista_ub}}", html_ub)
html = html.replace("{{fecha}}", str(hoy))
html = html.replace("{{vacantes_cubiertas_totales}}", str(vacantes_cubiertas_totales))
html = html.replace("{{Total_de_vacantes_actuales}}", str(total_vacantes))
html = html.replace("{{vacantes_nuevas_total}}", str(vacantes_nuevas_totales))
html = html.replace("{{vacantes_sinmov_totales}}", str(vacantes_sinmov_totales))

# Guardar el HTML con los valores ya insertados
with open("C:\\Users\\centro.control\\Documents\\reporte_tickets.html", "w", encoding="utf-8") as f:
    f.write(html)